In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
rupaths = list(Path('/home/nikolare/_projects/axolotl/axolotl24_shared_task').glob('data/*/*test.ru.tsv'))
print(rupaths)
p = next(iter(rupaths))
print(p)


[PosixPath('/home/nikolare/_projects/axolotl/axolotl24_shared_task/data/test/axolotl.test.ru.tsv')]
/home/nikolare/_projects/axolotl/axolotl24_shared_task/data/test/axolotl.test.ru.tsv


In [3]:
df = pd.read_csv(p, sep='\t')
df.example = df.example.str.strip()  # Some examples contain space as the last symbol, which make further merging incorrect
print(len(df), df.word.nunique(), df.example.nunique())
df.head()

2126 211 1990


,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,test_ru_0,мёрзлый,мёрзлый,merzlyj_khrrgztit4U,"разг., устар. очень холодный; ледяной","Мерзлая земля скажется, только руку приложи От...",NaN,old,old
1,test_ru_1,мёрзлый,мёрзлый,NaN,NaN,Пантелей Прокофьевич сунул Григорию мёрзлую ру...,NaN,new,new
2,test_ru_2,мёрзлый,мёрзлый,NaN,NaN,"Между домом и рельсами, за широкой мёрзлой луж...",NaN,new,new
3,test_ru_3,мёрзлый,мёрзлый,NaN,NaN,"— Да, холодно, должно быть. На полу мёрзлые та...",NaN,new,new
4,test_ru_4,мёрзлый,мёрзлый,NaN,NaN,"Когда мосты были прорваны, безоружные солдаты,...",NaN,new,new


# Generate wordforms for each lemma with Pymorphy2

In [4]:
df.word.drop_duplicates().to_csv('lemmas.tsv', sep='\t', index=False, header=None)
!wc lemmas.tsv

 211  211 2857 lemmas.tsv


In [5]:
SUMMERWSI_HOME='/home/nikolare/_projects/summer-wsi/retrieval_lscd/summer-wsi'

In [6]:
!python $SUMMERWSI_HOME/retrieval/modules/generate_forms_pymorphy.py lemmas.tsv forms.tsv
!cut -f 1 -d '_' forms.tsv|sort|uniq|wc

    211     211    2857


## Add additional wordforms present in the dataset

In [7]:
lemma2addforms = {}
with open('forms_add.tsv','r') as inp:
    for l in inp:
        ff = l.strip().split('\t')
        lemma2addforms[ff[0]] = ff[1:]

In [8]:
with open('forms.tsv','r') as inp, open('forms_final.tsv','w') as outp:
    for l in inp:
        ff = l.strip().split('\t')
        lemma = ff[0].split('_')[0]
        if lemma in lemma2addforms:                    
            ff.extend(lemma2addforms[lemma])
        print('\t'.join(ff), file=outp)

In [9]:
!diff forms.tsv forms_final.tsv

101c101
< шаманить_INFN	шаманив	шаманившему	шаманишь	шаманим	шаманившею	шаманите	шаманящей	шаманящим	шаманившей	шаманившим	шаманила	шаманит	шаманило	шамань	шаманя	шаманю	шаманил	шаманящую	шаманивши	шаманящему	шаманить	шаманившую	шаманящие	шаманившими	шаманившие	шаманящих	шаманившее	шаманящею	шаманящее	шаманящий	шаманившая	шаманьте	шаманящими	шаманят	шаманили	шаманящего	шаманящем	шаманивший	шаманящая	шаманившего	шаманившем	шаманивших
---
> шаманить_INFN	шаманив	шаманившему	шаманишь	шаманим	шаманившею	шаманите	шаманящей	шаманящим	шаманившей	шаманившим	шаманила	шаманит	шаманило	шамань	шаманя	шаманю	шаманил	шаманящую	шаманивши	шаманящему	шаманить	шаманившую	шаманящие	шаманившими	шаманившие	шаманящих	шаманившее	шаманящею	шаманящее	шаманящий	шаманившая	шаманьте	шаманящими	шаманят	шаманили	шаманящего	шаманящем	шаманивший	шаманящая	шаманившего	шаманившем	шаманивших	шаманем
191c191
< кадра_	кадра
---
> кадра_	кадра	кадрой	кадры
214c214
< хлебало_	хлебало
---
> хлебало_	хлебало	хлебала


# Find usages of the generated wordforms

In [10]:
assert df.example.str.contains('\n').sum()==0, 'examples contain newlines, cannot write as plain text'
with open('corpus.txt', 'w') as outp:
    for ex in df.example.dropna():
        print(ex, file=outp)
!head corpus.txt

Мерзлая земля скажется, только руку приложи От того нельзя, что земля мерзла. Мерзлой
Пантелей Прокофьевич сунул Григорию мёрзлую руку, сел на край лавки, запахивая полу тулупа, обходя взглядом Аксинью, пристывшую у люльки.
Между домом и рельсами, за широкой мёрзлой лужей, проходила дорога, по которой возили дрова и воду.
— Да, холодно, должно быть. На полу мёрзлые тараканы валяются. И мыши тоже помёрзли.
Когда мосты были прорваны, безоружные солдаты, московские жители, женщины с детьми, бывшие в обозе французов, — всё под влиянием силы инерции не сдавалось, а бежало вперед в лодки, в мёрзлую воду.
Мёрзлый пар валил с загнанных насмерть лошадей, с бегущих людей.
Я подошёл к окну, приложил лоб к мёрзлому стеклу и помню, что мне лоб обожгло льдом, как огнём.
Неужели и в уголку мёрзлой хижины не хочешь ты дать бедняку местечка?
Все больны, все бредят, кто хохочет, кто на стену лезет; в избах смрад, ни воды подать, ни принести её некому, а пищей служит один мёрзлый картофель.
Винтом крутит

In [11]:
!python $SUMMERWSI_HOME/retrieval/modules/regex_retriever.py forms_final.tsv corpus.txt usages

Compiled regexp (maybe shortened):
 re.compile('\\b(померзлее|мерзло|мёрзлого|мерзл|мёрзлых|мерзлая|помёрзлей|мёрзлою|мёрзлой|мерзлое|мерзла|мерзлый|мёрзлыми|мёрзлая|мёрзлому|мёрзлые|мёрзло|мерзлого|мёрзлый|мерзлой|мёрзлее|мерзлую|мёрзлым|мерзлее|, re.IGNORECASE) 

Full regexp:
 \b(померзлее|мерзло|мёрзлого|мерзл|мёрзлых|мерзлая|помёрзлей|мёрзлою|мёрзлой|мерзлое|мерзла|мерзлый|мёрзлыми|мёрзлая|мёрзлому|мёрзлые|мёрзло|мерзлого|мёрзлый|мерзлой|мёрзлее|мерзлую|мёрзлым|мерзлее|мерзлою|мерзлей|мерзлым|мерзлых|мёрзлую|мёрзла|мерзлом|мерзлому|мёрзлом|мерзлы|мерзлыми|померзлей|мерзлые|мёрзлы|мёрзлей|помёрзлее|мёрзлое|мёрзл|элемент|элементы|элемента|элементу|элементом|элементам|элементами|элементах|элементе|элементов|кабаках|кабаки|кабаками|кабаков|кабак|кабаке|кабаку|кабака|кабакам|кабаком|трубаче|трубачей|трубач|трубачи|трубачу|трубача|трубачом|трубачам|трубачами|трубачах|славнейшей|славнейшими|славное|пославнее|славнее|славнейшего|славную|славными|славна|славней|славен|славны|славным|славном

100%|███████████████████████████████| 478086/478086 [00:04<00:00, 101136.74it/s]
Converting retrieved usages to a DataFrame...

Usages found:
другой_ADJF        107
земля_NOUN          89
рука_NOUN           73
мелкий_ADJF         53
рыжий_ADJF          48
                  ... 
шпана_NOUN           3
тарабанить_INFN      3
топ_INTJ             1
слыхать_PRED         1
слыхать_CONJ         1
Name: word, Length: 221, dtype: int64

Usages found per form:
*** давиться_INFN ***
давится     6
давясь      2
давиться    2
давятся     2
давись      1
Name: wordat, dtype: int64
*** давление_NOUN ***
давления    6
давление    2
давлении    1
Name: wordat, dtype: int64
*** дека_NOUN ***
деки    3
дека    1
Дека    1
Name: wordat, dtype: int64
*** депо_NOUN ***
депо    10
Name: wordat, dtype: int64
*** дикий_ADJF ***
дикой     7
дикий     5
дикие     4
диких     4
Дикая     2
дикая     2
диким     2
дикое     2
дикую     2
дика      2
дикому    1
дики      1
диком     1
Дикие     1
дикими    1
Дик

*** матроска_NOUN ***
матроске     2
Матроска     1
матросках    1
матроска     1
Матроске     1
Name: wordat, dtype: int64
*** маячить_INFN ***
маячить    4
маячили    2
маячит     2
маячишь    1
маячил     1
маячила    1
Маячить    1
Name: wordat, dtype: int64
*** мебельный_ADJF ***
мебельный     3
мебельного    2
Мебельный     1
мебельное     1
мебельная     1
мебельной     1
мебельным     1
Name: wordat, dtype: int64
*** медвяный_ADJF ***
медвяный    4
медвяные    4
медвяною    2
медвяно     1
медвяных    1
медвяной    1
медвяным    1
медвяна     1
Name: wordat, dtype: int64
*** мелкий_ADJF ***
мелкие     8
мелкая     6
мелких     5
мелкий     5
мелким     4
мелко      3
мелкой     3
Мелкие     3
мелкую     2
Мелкий     2
мелкое     2
Мелкая     2
мелкими    2
Мелкому    1
Мелкою     1
мелкого    1
мелкому    1
мелки      1
Мелко      1
Name: wordat, dtype: int64
*** мелькать_INFN ***
мелькал     9
мелькает    3
мелькают    1
Name: wordat, dtype: int64
*** месть_NOUN ***
месть     

травля     3
травлю     1
травлей    1
травлею    1
Name: wordat, dtype: int64
*** трап_NOUN ***
трап      3
трапа     2
трапов    2
трапу     1
Трапы     1
Name: wordat, dtype: int64
*** трубач_NOUN ***
трубач      6
трубачей    3
трубачом    1
трубачи     1
Трубач      1
Name: wordat, dtype: int64
*** трёпка_NOUN ***
трёпки    5
трёпку    4
Трепка    1
трепку    1
Name: wordat, dtype: int64
*** тулуп_NOUN ***
тулуп     3
Тулуп     3
тулупе    2
тулупа    1
Name: wordat, dtype: int64
*** тупить_INFN ***
тупит     4
тупить    3
тупя      3
тупили    1
тупят     1
тупите    1
тупила    1
тупил     1
тупило    1
Name: wordat, dtype: int64
*** тяжесть_NOUN ***
тяжесть     12
тяжестью     7
тяжести      6
тяжестях     1
Name: wordat, dtype: int64
*** убрать_INFN ***
Убрать     6
убрать     5
убрали     4
убрала     3
убрал      2
Убрали     2
Уберите    1
убран      1
убрав      1
Name: wordat, dtype: int64
*** увязать_INFN ***
увязали       4
увязала       2
увязанными    1
увязать       

*** южный_ADJF ***
южной     5
южному    2
южного    2
Южный     1
южном     1
Южная     1
Южные     1
южный     1
Южным     1
южные     1
Name: wordat, dtype: int64
*** юр_NOUN ***
юру     10
юр       2
юра      1
юром     1
Юр       1
Name: wordat, dtype: int64
*** яблоко_NOUN ***
яблоки     6
яблоко     5
яблоком    3
яблока     3
яблок      1
Яблоко     1
Name: wordat, dtype: int64
*** явка_NOUN ***
явка    7
явки    3
Явка    2
явке    1
Явки    1
Name: wordat, dtype: int64
*** ягодица_NOUN ***
ягодицы     4
Ягодица     1
ягодице     1
ягодицам    1
Name: wordat, dtype: int64
*** ядрышко_NOUN ***
ядрышко    3
ядрышки    1
Name: wordat, dtype: int64
*** язва_NOUN ***
язва      4
язвы      3
Язва      2
язвами    1
язвою     1
язву      1
язв       1
Name: wordat, dtype: int64
*** язык_NOUN ***
язык      13
Язык       8
языком     8
языка      6
языке      5
языки      4
Языком     1
языку      1
Name: wordat, dtype: int64
*** ярлык_NOUN ***
ярлык     4
Ярлык     2
ярлыке    1
ярлык

# Load indices found

In [12]:
dfi = pd.concat([pd.read_csv(p, sep='\t') for p in Path('usages').glob('*.tsv')], ignore_index=True)
dfi.head()

,word,context_id,context,positions,gold_sense_id,predict_sense_id,wordat
0,округ_NOUN,corpus_3329_17,"Ветер шатался по округе много дней и ночей, вы...",17-23,-1,NaN,округе
1,округ_NOUN,corpus_560_174,В соответствии с принятой классификацией внутр...,174-180,-1,NaN,округа
2,округ_NOUN,corpus_557_107,Дикий-барин (так его прозвали; настоящее же ег...,107-113,-1,NaN,округе
3,округ_NOUN,corpus_558_94,"Но всё же заявление она Ризину не передала, а ...",94-100,-1,NaN,округа
4,округ_NOUN,corpus_5282_128,"Если Сахалин, ― как в шутку называют его местн...",128-133,-1,NaN,округ


In [13]:
dfi.word = dfi.word.str.split('_').str[0]

dfi = dfi.groupby(['word','context']).positions.agg(set).reset_index()
dfi.head()

,word,context,positions
0,аванпост,К пятому веку он [ Херсонес ] снова самое силь...,{202-210}
1,аванпост,"Проводник бежал, Суворов заблудился, проплутал...",{126-135}
2,аванпост,Пулемет Родиона Малиновского был установлен на...,{47-56}
3,аванпост,"Только в ГДР, на этом социалистическом аванпос...",{39-48}
4,агония,"Началась мучительная агония немецких войск, за...",{21-27}


In [14]:
ll = dfi.positions.apply(len)
ll.value_counts()

1     12964
2       839
3       246
4       137
5        57
6        42
8        21
7        17
9        17
10       11
11        8
12        7
13        5
15        5
18        4
16        4
20        2
19        2
14        2
23        2
32        1
31        1
63        1
42        1
Name: positions, dtype: int64

### If several occurrences found in the same example, select one of them wisely (with enough left and right context)

In [15]:
pd.set_option('colwidth',300)
dfi[ll==3]

,word,context,positions
252,безвременье,"Время красит, безвременье старит. Было время, осталось одно безвременье. Безвременье придет -- все добро как мылом возьмет.","{14-25, 60-71, 73-84}"
271,беззубый,"Черепаха беззубое животное. Беззубая старуха. Беззубая борона, вилка,","{46-54, 28-36, 9-17}"
276,безоружный,"С безоружными жителями не воюют. Заяц безоружное животное. Безоружным глазом не увидишь ни спутников планетных, ни мельчайших наливняков.","{2-13, 59-69, 38-48}"
341,бич,"Прикочевавшие к теплу советские бичи прочно заняли в сознании и обществе чеченцев место лайев, пришлых батраков. 〈…〉 Когда бичи жили при хозяевах, то милиция их не трогала — каждый селянин был милиционеру друг или родственник. 〈…〉 Колька был особенный бич. Прежде всего он не пил.","{32-36, 123-127, 252-255}"
396,блюдо,"За столом было двенадцать блюд или перемен. До французских блюд я не охотник. За спесивым кумом не находишься с блюдом, потчуя. Тот же блин, да на","{26-30, 59-63, 112-118}"
...,...,...,...
13972,штука,"Штука обеденный, банкетный стол, складной, раскидной ипр. Штука с полами и штука раздвижная на 24 прибора.","{75-80, 58-63, 0-5}"
14028,щекотать,"юж., щелоктать раждать особое чувство раздраженья в теле, в коже, касаясь её или перебирая пальцами. Детей никогда не щекочи, это шутка вредная. Шипучка инно в ногу щекочет, защекотала. Зуд щекочет. Щекотни-ка его под бока!","{118-124, 165-172, 190-197}"
14165,юр,"На мельнице юр-юром, завозно, много народу. Лавка на юру, на самом углу рынка.","{15-19, 12-14, 53-56}"
14299,язык,"Язык колокола, [или в колоколе, то, что в нем звучит, действует], било, клепало, железный пест, кистень, привешиваемый внутри под шелом колокола, для звону. Ямщики любят подвязывать в колокольчик кольцо вместо языка. Без языка и колокол","{210-215, 221-226, 0-4}"


In [16]:
def select_occurrence(pp, context):
    if len(pp) == 1:
        return next(iter(pp))

    pps = sorted([list(map(int, p.split('-'))) for p in  pp], key=lambda p: p[0])
    if len(pps) == 2:
        # when two occurrences, select the one which has longer context both to the right and to the left; 
        # short left or right context (especially when one of them is empty) may be a problem for some Transformer heads
        left = pps[0][0] 
        right = len(context) - pps[1][1]
        return '-'.join( map(str, pps[0] if left > right else pps[1]) )
        
    # the heuristic takes second to last occurrence, it has the longest left context
    # and also some right context
    return '-'.join(map(str,pps[-2]))
    

In [17]:
dfi['positions'] = dfi.apply(lambda r: select_occurrence(r.positions, r.context), axis=1)

In [18]:
dfi[ll==2]

,word,context,positions
44,ангажировать,"Тебя местные органы, небось, уже на полную катушку ангажировали, а? — Славка протянула «офицеру» свой пустой стакан. — Они его ангажировали от слова «выжимать».",51-63
64,антихрист,"Раскол был уходом из истории, потому что историей овладел князь этого мира, антихрист, проникший на вершины церкви и государства. 〈…〉 Уход из государства оправдывался тем, что в нём не было правды, торжествовал не Христос, а антихрист.",76-85
70,аристократ,"Ему хочется попасть в аристократы, он лезет в знать. Во всяком сословии и звании могуть быть своего рода аристократы,",22-33
87,артерия,"артерии несут алую кровь из сердца по всем частям тела, откуда она, через тончайшие волосные сосуды, возвращается венами это большой кровеоборот, питания; артерии же, или боевые жилы, проводят обращенную в сердце чорную кровь в легкия, откуда она возвращается в сердце венами; это малый кровеобор...",155-162
99,аршин,"Водку пили не рюмками, а «аршинами», и нужно было выпить не менее аршина рюмок, поставленных в ряд.",66-72
...,...,...,...
14221,явка,"Плейшнер пошёл к дому, где помещалась явка, и, взглянув в окно, увидел высокого хозяина явки и черноволосого.",38-42
14272,язык,"И вот, вы и не успеете заметить, как язык хамелеона, молниеносным движением выброшенный вперёд, уже возвращается с прилипшей к языку добычей.",37-41
14379,ясный,"Ясные с тёмной поволокой глаза молодой вдовы были очень мало заплаканы, и чуть только она со свёкром выехала с кладбища на поле, отделяющее могилки от города, эти ясные глаза совсем высохли и взглянули из-под густых ресниц своих ещё чище, чем смотрели доселе.",163-168
14386,ястреб,"ястреб голубятник, тетеревятник, тетерник и кобчик, перепелятник; сарыча и балабана инде также зовут ястребом,",101-109


## Add positions to the main dataset

In [19]:
dfi = dfi.rename(columns={'context':'example'})

In [20]:
dfm = df.merge(dfi, on=['word','example'], how='left')
len(df), len(dfm)

(2126, 2126)

In [21]:
# old examples are glosses, which often do not target word
dfm[dfm.positions.isnull()].period.value_counts()

old    170
new      1
Name: period, dtype: int64

In [23]:
# old examples are glosses, which often do not target word, we don't need positions for them; look at the new ones
dfm[dfm.positions.isnull() & (dfm.period=='new')]

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period,positions
1122,test_ru_1126,королёк,королёкъ,NaN,NaN,ъ,NaN,new,new,NaN


# Now add wordforms manually to forms_add.csv...
Based on the table above, we added wordforms manually to forms_add.csv and reran the notebook from the beginning.
Those usages that do not contain the target word are errors in the original dataset.

In [29]:
# For our methods, we didn't need positions of the target word for the old usage. We saved time and didn't
# try to add additional wordforms for the old usages. But dumped them to fix in the future.
dfm[dfm.positions.isnull() & ~dfm.example.isnull() & (dfm.period=='old')][['usage_id','word','example']].to_csv(
    'partially_errors.tsv',sep='\t',index=False)

In [23]:
# This usage from the test set is an error, we didn't want to add this to our wordforms; 
# however, we cannot drop examples from the test set, so just set some indices
dfm.loc[dfm.usage_id=='test_ru_1126','indices_target_token'] = '0-1'

# Save the dataframe with recovered indices

In [24]:
dfm['indices_target_token'] = dfm.positions.str.split('-').str.join(':')
dfr = dfm.drop(columns=['positions'])
dfr.head()

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,test_ru_0,мёрзлый,мёрзлый,merzlyj_khrrgztit4U,"разг., устар. очень холодный; ледяной","Мерзлая земля скажется, только руку приложи От того нельзя, что земля мерзла. Мерзлой",70:76,old,old
1,test_ru_1,мёрзлый,мёрзлый,NaN,NaN,"Пантелей Прокофьевич сунул Григорию мёрзлую руку, сел на край лавки, запахивая полу тулупа, обходя взглядом Аксинью, пристывшую у люльки.",36:43,new,new
2,test_ru_2,мёрзлый,мёрзлый,NaN,NaN,"Между домом и рельсами, за широкой мёрзлой лужей, проходила дорога, по которой возили дрова и воду.",35:42,new,new
3,test_ru_3,мёрзлый,мёрзлый,NaN,NaN,"— Да, холодно, должно быть. На полу мёрзлые тараканы валяются. И мыши тоже помёрзли.",36:43,new,new
4,test_ru_4,мёрзлый,мёрзлый,NaN,NaN,"Когда мосты были прорваны, безоружные солдаты, московские жители, женщины с детьми, бывшие в обозе французов, — всё под влиянием силы инерции не сдавалось, а бежало вперед в лодки, в мёрзлую воду.",183:190,new,new


In [25]:

dfr.to_csv('fixedindices_'+p.name, sep='\t', index=False)


# Some auxiliary inspection

In [38]:
dfm.query('date=="old"').groupby('word').usage_id.nunique()

word
давиться    1
давление    2
дека        2
депо        1
дикий       4
           ..
ядрышко     1
язва        1
язык        8
ярлык       2
ясный       1
Name: usage_id, Length: 211, dtype: int64

In [43]:
dfm.query('word=="язык"').query('date=="old"')

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period,positions
1567,test_ru_1571,язык,языкъ,jazyk_oQqlJWJDoFM,"продольная палочка с зарубкою, с коей срывается отвесная подставка, сторожок","Язык в ловушке, насторожка, язычок",NaN,old,old,0-4
1573,test_ru_1577,язык,языкъ,jazyk_H1RadLJXL04,"било, которое ходит между щеками её",Язык мялицы,NaN,old,old,0-4
1576,test_ru_1580,язык,языкъ,jazyk_Z3SGNzPN1mg,"перен. подвесная деталь колокола (звонка, колокольчика и т. п.), как правило, в виде металлического стержня, которая, ударяясь о стенку, производит звук","Язык колокола, [или в колоколе, то, что в нем звучит, действует], било, клепало, железный пест, кистень, привешиваемый внутри под шелом колокола, для звону. Ямщики любят подвязывать в колокольчик кольцо вместо языка. Без языка и колокол",NaN,old,old,210-215
1579,test_ru_1583,язык,языкъ,jazyk_S0YtaHtThcY,"анат. подвижный мускулистый орган в ротовой полости позвоночных животных и человека, служащий для определения вкуса, захватывания, пережёвывания и глотания пищи, а у человека также для артикуляции речи","Коровий язык, лизун; рыбий, тумак; змеиный, жало, вилка; песий, лопата; кошачий, терка.",NaN,old,old,8-12
1582,test_ru_1586,язык,языкъ,jazyk_yPqs_d_x5C4,"пружинка, крючок, по коему бьют играя","Язык, язычок органа",NaN,old,old,0-4
1583,test_ru_1587,язык,языкъ,jazyk_PNFSmmmTmhk,"арх. слово из Архангельской губернии подводная ледяная коса, от тороса",NaN,NaN,old,old,NaN
1591,test_ru_1595,язык,языкъ,jazyk_RpvHYZLUoTs,"нутреная воронка мережи, вентера, через которую рыба входит детыш, детинец, ушинок",NaN,NaN,old,old,NaN
1595,test_ru_1599,язык,языкъ,jazyk_vM2JZPy1_oU,"пищик, тонкая деревянная пластинка, вставляемая в губник, для звука","Язык кларнета, гобоя",NaN,old,old,0-4
